In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [2]:
dirPath = "/Users/redpal/Desktop/data_science/overwatch_project"
os.chdir(dirPath)
%ls

damage_data.csv      support_data.csv     test_train/
data_split.zip       tank_data.csv
final_cleanup.ipynb  tank_train.csv


In [3]:
#Main Clean
def main_clean(inp_df, percentNaCol, percentNaRow, verbose=False):
    #Do Tank Data and Drop First Column
    percentNaRow = 1 - percentNaRow
    inp_df = inp_df.drop(columns=inp_df.columns[0],axis=1)

    #Verbose Statement
    if verbose:
        print("Prev Row: ",len(inp_df.index))

    #Strip Rows Based on Percent of NAs and No Na Ratins
    inp_df = inp_df[(inp_df.isna().sum(1)/inp_df.shape[1]).gt(percentNaRow)]
    inp_df = inp_df[inp_df['rating'].notna()]

    #Verbose Statement
    if verbose:
        print("Post Rows: ",len(inp_df.index))
        print("Prev Columns: ", len(inp_df.columns))

    #Strip Columns based on Percent of NAs
    for col in inp_df.columns:
        percentNa = int(inp_df[col].isnull().sum() / len(inp_df))
        if percentNa >= percentNaCol:
            del inp_df[col]

    #Verbose Statements
    if verbose:
        print("Post Columns: ", len(inp_df.columns))
        
    #Dtype Object Names
    groupByType = inp_df.columns.to_series().groupby(inp_df.dtypes).groups
    getName = {k.name: v for k, v in groupByType.items()}
    objNames = getName['object'].tolist()

    #Verbose Statement
    if verbose:
        print("Prev Col Types: ",list(groupByType.keys()))

    #Strip Percentage
    accAndEne = [x for x in objNames if "Acc" in x or "Ene" in x or "Percent" in x]
    for col in accAndEne:
        inp_df[col] = inp_df[col].map(lambda x: float(str(x).replace("%",""))/100)

    #Get Minutes to Seconds
    minutes = [x for x in objNames if "Min" in x]
    for col in minutes:
        df1 = inp_df[col].str.split(":", expand=True).astype(float)
        inp_df[col] = df1[0] * 60 + df1[1]

    #Sanity Check
    if verbose:
        print("Minute Cols: {} Percent Cols: {} All Object Cols: {} ".format(len(minutes),len(accAndEne),len(objNames)))
        altGroupBy = inp_df.columns.to_series().groupby(inp_df.dtypes).groups
        print("New Col Types:",list(altGroupBy.keys()))

    #Split into Test and Train
    train, test = train_test_split(inp_df, test_size=0.1)
    if verbose:
        print("Train Length: ", len(train.index)," Test Length: ",len(test.index))
    return train, test

In [4]:
#Variable
inp_folder = ""
final_folder = "test_train/"

#Get Tank_Df and Save
print("Tank")
tank_df = pd.read_csv(inp_folder+"tank_data.csv")
train, test = main_clean(tank_df,1.0,1.0,verbose=True)
train.to_csv(final_folder+"tank_train.csv",index=False)
test.to_csv(final_folder+"tank_test.csv",index=False)

#Get Support_Df and Save
print("\nSupport")
supp_df = pd.read_csv(inp_folder+"support_data.csv")
train, test = main_clean(supp_df,1.0,1.0,verbose=True)
train.to_csv(final_folder+"support_train.csv",index=False)
test.to_csv(final_folder+"support_test.csv",index=False)

#Get Damage_Df and Save
print("\nDamage")
damag_df = pd.read_csv(inp_folder+"damage_data.csv")
train, test = main_clean(damag_df,1.0,1.0,verbose=True)
train.to_csv(final_folder+"damage_train.csv",index=False)
test.to_csv(final_folder+"damage_test.csv",index=False)

Tank
Prev Row:  20360
Post Rows:  20360
Prev Columns:  129
Post Columns:  120
Prev Col Types:  [dtype('float64'), dtype('O')]
Minute Cols: 16 Percent Cols: 10 All Object Cols: 26 
New Col Types: [dtype('float64')]
Train Length:  18324  Test Length:  2036

Support
Prev Row:  19800
Post Rows:  19800
Prev Columns:  127
Post Columns:  121
Prev Col Types:  [dtype('float64'), dtype('O')]
Minute Cols: 14 Percent Cols: 10 All Object Cols: 24 
New Col Types: [dtype('float64')]
Train Length:  17820  Test Length:  1980

Damage
Prev Row:  18270
Post Rows:  18270
Prev Columns:  270
Post Columns:  264
Prev Col Types:  [dtype('float64'), dtype('O')]
Minute Cols: 34 Percent Cols: 38 All Object Cols: 72 
New Col Types: [dtype('float64')]
Train Length:  16443  Test Length:  1827
